In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">Instacart Market Basket Feature Engineering</h3>
    <p><br>This notebook will be related to Feature Engineering, which is the stage of creating new variables to help the model make better predictions later.</p>
</div>

In [6]:
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import tensorflow as tf
import polars as pl
import duckdb
import sys

In [7]:
path = '/kaggle/input/datasets/marcin123321/function2/' 
sys.path.append(path) 
from DUCKDB_2 import *

In [8]:
%cd /kaggle/working

/kaggle/working


In [9]:
!cp /kaggle/input/datasets/marcin123321/function2/DUCKDB_2.py /kaggle/working/

In [10]:
!cp /kaggle/input/order-new-parquet/orders_new.parquet /kaggle/working/

<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">Create basket size, meaning how many products the order consists of</h3>
</div>

In [11]:
create_basket_size(con)
con.execute("SELECT * FROM 'basket_size.parquet' LIMIT 5").df()

,order_id,basket_size
0,2,9
1,3,8
2,4,13
3,5,26
4,6,3


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">Merging basket size and orders_new</h3>
</div>

In [12]:
create_table_orders_bs(con)
con.execute("SELECT * FROM 'orders_bs.parquet' LIMIT 5").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_id_1,basket_size
0,2295428,13475,prior,8,1,10,16.0,32433,4,1,2295428,10
1,2295428,13475,prior,8,1,10,16.0,13909,5,1,2295428,10
2,2295428,13475,prior,8,1,10,16.0,29127,6,0,2295428,10
3,2295428,13475,prior,8,1,10,16.0,8424,7,1,2295428,10
4,2295428,13475,prior,8,1,10,16.0,20985,8,1,2295428,10


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">For cumcount to work correctly, we sort the data by the appropriate columns </h3>
</div>


In [13]:
create_user_product_orders(con)
con.execute("SELECT * FROM 'user_product_orders.parquet' LIMIT 5").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id
0,2539329,1,1,2,8,0,NaN,1,196
1,2398795,1,2,3,7,1,15.0,1,196
2,473747,1,3,3,12,1,21.0,1,196
3,2254736,1,4,4,7,1,29.0,1,196
4,431534,1,5,4,15,1,28.0,1,196


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">So it adds a column showing how many times the user has bought this product before</h3>
</div>


In [14]:
create_user_product_with_counts(con)
con.execute("SELECT * FROM 'upo_step1.parquet' LIMIT 15").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far
0,1830137,9,3,5,12,0,30.0,18,10132,0
1,1830137,9,3,5,12,0,30.0,19,24954,0
2,1166994,21,3,1,13,0,7.0,2,14788,0
3,1674891,21,18,2,11,1,21.0,3,14788,1
4,1413723,21,20,1,10,1,12.0,3,14788,2
5,865324,21,32,1,13,1,12.0,3,14788,3
6,2064090,29,1,5,11,0,NaN,5,36543,0
7,875452,29,2,2,8,1,11.0,5,36543,1
8,2657652,29,3,2,10,1,14.0,1,36543,2
9,2999889,29,4,6,8,1,11.0,1,36543,3


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">For each user and product, it shifts the purchase history down by one row</h3>
</div>



In [15]:
create_upo_step2(con)
con.execute("SELECT * FROM 'upo_step2.parquet' LIMIT 15").df()
#user_product['last_order_number'] = (
#    user_product
#    .groupby(['user_id', 'product_id'])
#    ['order_number']
#    .shift(1)
#)
#user_product.head(15)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far,last_order_number
0,2539329,1,1,2,8,0,NaN,1,196,0,<NA>
1,2398795,1,2,3,7,1,15.0,1,196,1,1
2,473747,1,3,3,12,1,21.0,1,196,2,2
3,2254736,1,4,4,7,1,29.0,1,196,3,3
4,431534,1,5,4,15,1,28.0,1,196,4,4
5,3367565,1,6,2,7,1,19.0,1,196,5,5
6,550135,1,7,1,9,1,20.0,1,196,6,6
7,3108588,1,8,1,14,1,14.0,2,196,7,7
8,2295261,1,9,1,16,1,0.0,4,196,8,8
9,2550362,1,10,4,8,1,30.0,1,196,9,9


In [16]:
create_upo_step3(con)
con.execute("SELECT * FROM 'upo_step3.parquet' LIMIT 15").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far,last_order_number,orders_since_last_purchase
0,2539329,1,1,2,8,0,NaN,1,196,0,<NA>,0
1,2398795,1,2,3,7,1,15.0,1,196,1,1,1
2,473747,1,3,3,12,1,21.0,1,196,2,2,1
3,2254736,1,4,4,7,1,29.0,1,196,3,3,1
4,431534,1,5,4,15,1,28.0,1,196,4,4,1
5,3367565,1,6,2,7,1,19.0,1,196,5,5,1
6,550135,1,7,1,9,1,20.0,1,196,6,6,1
7,3108588,1,8,1,14,1,14.0,2,196,7,7,1
8,2295261,1,9,1,16,1,0.0,4,196,8,8,1
9,2550362,1,10,4,8,1,30.0,1,196,9,9,1


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">Average of days_since_prior_order calculated in a moving window over the user's previous orders (rolling average without future data)</h3>
</div>

In [19]:
create_upo_step5(con)
con.execute("SELECT * FROM 'upo_step5.parquet' LIMIT 15").df()

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far,last_order_number,orders_since_last_purchase,user_avg_days_between_orders_so_far
0,475282,53126,7,4,13,1,15.0,7,25647,1,2,5,19.988095
1,475282,53126,7,4,13,1,15.0,4,25890,1,2,5,19.929412
2,475282,53126,7,4,13,0,15.0,2,26604,0,<NA>,0,19.872093
3,475282,53126,7,4,13,0,15.0,1,34584,0,<NA>,0,19.816092
4,475282,53126,7,4,13,1,15.0,3,37646,1,2,5,19.761364
5,475282,53126,7,4,13,0,15.0,9,45066,0,<NA>,0,19.707865
6,475282,53126,7,4,13,1,15.0,10,46886,2,6,1,19.655556
7,3061720,53126,8,2,13,0,30.0,3,781,0,<NA>,0,19.604396
8,3061720,53126,8,2,13,1,30.0,11,4920,1,6,2,19.717391
9,3061720,53126,8,2,13,1,30.0,15,5212,2,7,1,19.827957


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">Number of products in a given order for a given user</h3>
</div>

In [20]:
create_orders_bs(con)
con.execute("SELECT * FROM 'orders_bs.parquet' LIMIT 5").df()


,user_id,order_id,basket_size
0,1666,900824,11
1,1666,3358592,8
2,1670,1026034,17
3,1683,3206875,4
4,1686,1017881,2


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">Rolling average of the basket size size from previous orders (excluding the future)</h3>
</div>

In [21]:
create_upo_step6(con)
con.execute("SELECT * FROM 'upo_step6.parquet' LIMIT 15").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far,last_order_number,orders_since_last_purchase,user_avg_days_between_orders_so_far,user_avg_basket_size_so_far
0,1989267,172733,19,0,15,0,5.0,22,46526,0,<NA>,0,14.857143,12.831683
1,1989267,172733,19,0,15,1,5.0,18,48670,1,17,2,14.805263,12.891626
2,1989267,172733,19,0,15,1,5.0,14,49517,3,15,4,14.753927,12.950980
3,1989267,172733,19,0,15,1,5.0,7,4292,1,2,17,14.703125,13.009756
4,1989267,172733,19,0,15,1,5.0,15,5258,4,18,1,14.652850,13.067961
5,1989267,172733,19,0,15,0,5.0,10,5621,0,<NA>,0,14.603093,13.125604
6,1989267,172733,19,0,15,1,5.0,16,7049,1,6,13,14.553846,13.182692
7,1989267,172733,19,0,15,0,5.0,12,7751,0,<NA>,0,14.505102,13.239234
8,1989267,172733,19,0,15,1,5.0,19,8424,1,16,3,14.456853,13.295238
9,1989267,172733,19,0,15,0,5.0,3,8565,0,<NA>,0,14.409091,13.350711


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">How many times has this product been purchased before</h3>
</div>


In [22]:
create_upo_step7(con)
con.execute("SELECT * FROM 'upo_step7.parquet' LIMIT 15").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far,last_order_number,orders_since_last_purchase,user_avg_days_between_orders_so_far,user_avg_basket_size_so_far,product_total_purchases_so_far
0,2353587,95146,2,2,3,0,16.0,10,1123,0,<NA>,0,NaN,11.000000,0
1,1785558,157392,2,1,12,0,30.0,15,1123,0,<NA>,0,30.000000,36.605263,1
2,304751,50940,2,1,8,0,30.0,16,1123,0,<NA>,0,30.000000,11.416667,2
3,1134718,157392,3,5,16,1,18.0,3,1123,1,2,1,29.000000,31.360656,3
4,3015434,50940,3,6,21,1,30.0,3,1123,1,2,1,30.000000,13.962963,4
5,590266,11134,4,4,16,0,30.0,15,1123,0,<NA>,0,20.571429,19.145455,5
6,3275422,51823,5,5,13,0,11.0,17,1123,0,<NA>,0,23.276923,24.565217,6
7,1127466,32494,20,0,12,0,12.0,17,1123,0,<NA>,0,10.437229,14.951220,7
8,2306464,196085,54,5,10,0,3.0,15,1123,0,<NA>,0,6.517799,16.691574,8
9,1321266,196085,56,2,8,1,6.0,15,1123,1,54,2,6.375191,16.753012,9


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">How many different users bought the product earlier</h3>
</div>



In [23]:
create_upo_step8(con)
con.execute("SELECT * FROM 'upo_step8.parquet' LIMIT 15").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far,last_order_number,orders_since_last_purchase,user_avg_days_between_orders_so_far,user_avg_basket_size_so_far,product_total_purchases_so_far,product_unique_users_so_far
0,2539526,200460,6,5,9,0,5.0,7,21137,0,<NA>,0,5.966102,15.970588,82434,42543
1,610779,203310,6,6,1,0,15.0,13,21137,0,<NA>,0,12.417910,20.375000,82435,42544
2,1913549,115610,6,0,16,0,10.0,25,21137,0,<NA>,0,14.100000,26.658915,82436,42545
3,29164,119080,6,5,12,0,7.0,3,21137,0,<NA>,0,12.933333,15.361111,82437,42546
4,677245,146829,6,2,15,0,5.0,2,21137,0,<NA>,0,7.176471,5.000000,82438,42547
5,1860963,150914,6,6,15,0,8.0,13,21137,0,<NA>,0,6.674699,18.900990,82439,42548
6,2246435,196314,6,2,8,1,5.0,3,21137,3,4,2,4.047619,9.354167,82440,42549
7,2310643,198495,6,5,15,1,17.0,10,21137,1,1,5,21.525000,10.358491,82441,42549
8,415475,134084,6,3,21,1,5.0,7,21137,2,5,1,4.756098,11.232143,82442,42549
9,1466482,144423,6,6,14,1,30.0,1,21137,1,1,5,25.416667,3.625000,82443,42549


<div style="
    border: 2px solid black;
    background: #e8e8ff;
    padding: 20px;
    border-radius: 12px;
    font-family: 'Arial'; 
    font-size: 20px;
">
    <h3 style="margin: 0;
        font-size: 35px;">Rolling average reorder rate for a user–product pair from previous orders</h3>
</div>




In [25]:
create_upo_final(con)
con.execute("SELECT * FROM 'upo_final.parquet' LIMIT 15").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,order_id,user_id,order_number,order_dow,order_hour_of_day,reordered,days_since_prior_order,add_to_cart_order,product_id,times_user_bought_product_so_far,last_order_number,orders_since_last_purchase,user_avg_days_between_orders_so_far,user_avg_basket_size_so_far,product_total_purchases_so_far,product_unique_users_so_far,user_product_reorder_rate_so_far
0,1247527,12,2,5,9,0,30.0,4,7076,0,<NA>,0,30.000000,6.500000,183,142,NaN
1,1916106,12,3,5,8,1,14.0,9,7076,1,2,1,29.111111,14.761905,270,183,0.000000
2,221248,12,5,1,9,1,30.0,14,7076,2,3,2,24.560000,16.301887,454,268,0.500000
3,2610576,25,1,1,14,0,NaN,11,31225,0,<NA>,0,NaN,11.000000,74,55,NaN
4,484397,37,6,5,17,0,10.0,5,20995,0,<NA>,0,7.421053,10.420000,9824,6328,NaN
5,2681913,46,18,0,9,0,22.0,13,24660,0,<NA>,0,17.421488,8.803030,515,237,NaN
6,2009771,46,19,0,16,1,21.0,5,24660,1,18,1,17.866667,9.397260,532,243,0.000000
7,1657641,48,6,5,9,0,27.0,5,34012,0,<NA>,0,25.448980,10.843137,731,458,NaN
8,817409,61,1,2,11,0,NaN,16,25970,0,<NA>,0,NaN,30.000000,2,2,NaN
9,390867,63,16,3,15,0,5.0,12,27104,0,<NA>,0,11.102564,18.736000,41684,20895,NaN
